In [1]:
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sys, cv2, glob, os, time
import pandas as pd 
import io
import numpy as np
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten,Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
import numpy as np # linear algebra

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import os
import glob
import cv2
import tensorflow as tf
from keras import layers
from keras.layers import Dropout , Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from sklearn.model_selection import train_test_split
import keras.backend as K
from sklearn.utils import shuffle
# importing all necessary libraries to run the code
import re,string
import numpy as np
import pandas as pd
import keras_metrics
import tensorflow.keras
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,Embedding
# using the variable sw to hold all stopwords that are in English
sw = stopwords.words('english')
import matplotlib.pyplot as plt

In [2]:
dataUN=pd.read_csv('UNSW_NB15_testing-set.csv')
dataUN1=pd.read_csv('UNSW_NB15_training-set.csv')

dataI=pd.read_csv('IoT Network Intrusion Dataset.csv')

In [3]:
dataUN1[:2]

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0


In [4]:
dataI[:2]

,Flow_ID,Src_IP,Src_Port,Dst_IP,Dst_Port,Protocol,Timestamp,Flow_Duration,Tot_Fwd_Pkts,Tot_Bwd_Pkts,...,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label,Cat,Sub_Cat
0,192.168.0.13-192.168.0.16-10000-10101-17,192.168.0.13,10000,192.168.0.16,10101,17,25/07/2019 03:25:53 AM,75,1,1,...,0.0,0.0,0.0,75.0,0.000000,75.0,75.0,Anomaly,Mirai,Mirai-Ackflooding
1,192.168.0.13-222.160.179.132-554-2179-6,222.160.179.132,2179,192.168.0.13,554,6,26/05/2019 10:11:06 PM,5310,1,2,...,0.0,0.0,0.0,2655.0,2261.327486,4254.0,1056.0,Anomaly,DoS,DoS-Synflooding


In [5]:
dataF=dataUN.append(dataUN1,ignore_index = True) 

In [6]:
dataF[:2]

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0


In [7]:
from collections import Counter
Counter(dataI["Sub_Cat"])

Counter({'Mirai-Ackflooding': 55124,
         'DoS-Synflooding': 59391,
         'Scan Port OS': 53073,
         'Mirai-Hostbruteforceg': 121181,
         'Mirai-UDP Flooding': 183554,
         'Mirai-HTTP Flooding': 55818,
         'Normal': 40073,
         'Scan Hostport': 22192,
         'MITM ARP Spoofing': 35377})

In [8]:
from collections import Counter
Counter(dataF["attack_cat"])

Counter({'Normal': 93000,
         'Backdoor': 2329,
         'Analysis': 2677,
         'Fuzzers': 24246,
         'Shellcode': 1511,
         'Reconnaissance': 13987,
         'Exploits': 44525,
         'DoS': 16353,
         'Worms': 174,
         'Generic': 58871})

In [9]:
from collections import Counter
Counter(dataI["Label"])

Counter({'Anomaly': 585710, 'Normal': 40073})

In [10]:
dataF1=dataF.loc[dataF['attack_cat'] == "Normal"]
print(len(dataF1))
dataF2=dataF.loc[dataF['attack_cat'] == "Backdoor"]
print(len(dataF2))
dataF3=dataF.loc[dataF['attack_cat'] == "Analysis"]
print(len(dataF3))
dataF4=dataF.loc[dataF['attack_cat'] == "Fuzzers"]
print(len(dataF4))
dataF5=dataF.loc[dataF['attack_cat'] == "Shellcode"]
print(len(dataF5))
dataF6=dataF.loc[dataF['attack_cat'] == "Reconnaissance"]
print(len(dataF6))
dataF7=dataF.loc[dataF['attack_cat'] == "Exploits"]
print(len(dataF7))
dataF8=dataF.loc[dataF['attack_cat'] == "DoS"]
print(len(dataF8))
dataF9=dataF.loc[dataF['attack_cat'] == "Worms"]
print(len(dataF9))
dataF10=dataF.loc[dataF['attack_cat'] == "Generic"]
print(len(dataF10))



dataI1=dataI.loc[dataI['Sub_Cat'] == 'Mirai-Ackflooding']
print(len(dataI1))
dataI2=dataI.loc[dataI['Sub_Cat'] == 'DoS-Synflooding']
print(len(dataI2))
dataI3=dataI.loc[dataI['Sub_Cat'] == 'Scan Port OS']
print(len(dataI3))
dataI4=dataI.loc[dataI['Sub_Cat'] == 'Mirai-Hostbruteforceg']
print(len(dataI4))

dataI5=dataI.loc[dataI['Sub_Cat'] == 'Mirai-UDP Flooding']
print(len(dataI5))
dataI6=dataI.loc[dataI['Sub_Cat'] == 'Mirai-HTTP Flooding']
print(len(dataI6))

dataI7=dataI.loc[dataI['Sub_Cat'] == 'Scan Hostport']
print(len(dataI7))
dataI8=dataI.loc[dataI['Sub_Cat'] == 'MITM ARP Spoofing']
print(len(dataI8))

dataI9=dataI.loc[dataI['Sub_Cat'] == 'Normal']
print(len(dataI9))



93000
2329
2677
24246
1511
13987
44525
16353
174
58871
55124
59391
53073
121181
183554
55818
22192
35377
40073


In [11]:
dataF1=dataF1[0:170]
dataF2=dataF2[0:170]
dataF3=dataF3[0:170]
dataF4=dataF4[0:170]
dataF5=dataF5[0:170]
dataF6=dataF6[0:170]
dataF7=dataF7[0:170]
dataF8=dataF8[0:170]
dataF9=dataF9[0:170]
dataF10=dataF10[0:170]


dataI1=dataI1[0:170]
dataI2=dataI2[0:170]
dataI3=dataI3[0:170]
dataI4=dataI4[0:170]
dataI5=dataI5[0:170]
dataI6=dataI6[0:170]
dataI7=dataI7[0:170]
dataI8=dataI8[0:170]
dataI9=dataI9[0:170]


In [12]:
dataF=dataF1.append(dataF2,ignore_index = True)
dataF=dataF.append(dataF3,ignore_index = True) 
dataF=dataF.append(dataF4,ignore_index = True) 
dataF=dataF.append(dataF5,ignore_index = True) 
dataF=dataF.append(dataF6,ignore_index = True) 
dataF=dataF.append(dataF7,ignore_index = True) 
dataF=dataF.append(dataF8,ignore_index = True) 
dataF=dataF.append(dataF9,ignore_index = True) 
dataF=dataF.append(dataF10,ignore_index = True) 

dataI=dataI1.append(dataI2,ignore_index = True)
dataI=dataI.append(dataI3,ignore_index = True)
dataI=dataI.append(dataI4,ignore_index = True)
dataI=dataI.append(dataI5,ignore_index = True)
dataI=dataI.append(dataI6,ignore_index = True)
dataI=dataI.append(dataI7,ignore_index = True)
dataI=dataI.append(dataI8,ignore_index = True)
dataI=dataI.append(dataI9,ignore_index = True)

In [13]:
from collections import Counter
Counter(dataI["Label"])

Counter({'Anomaly': 1360, 'Normal': 170})

In [14]:
from collections import Counter
Counter(dataF["label"])

Counter({0: 170, 1: 1530})

In [15]:
dataF.rename(columns = {'label':'Label'}, inplace = True)
dataF.rename(columns = {'attack_cat':'Sub_Cat'}, inplace = True)

In [16]:
DataITarget=dataI[["Label","Cat","Sub_Cat"]]

dataIFeatures=dataI.drop(["Label","Cat","Sub_Cat"], axis=1)

DataFTarget=dataF[["Sub_Cat","Label"]]

dataFFeatures=dataF.drop(["Sub_Cat","Label"], axis=1)

In [17]:
DataITarget["Sub_Cat"]=DataITarget["Sub_Cat"].replace('Normal','Normal_I')

C:\Users\SOHAIL~1\AppData\Local\Temp/ipykernel_17632/4285179342.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataITarget["Sub_Cat"]=DataITarget["Sub_Cat"].replace('Normal','Normal_I')


In [18]:
from collections import Counter
Counter(DataITarget["Sub_Cat"])

Counter({'Mirai-Ackflooding': 170,
         'DoS-Synflooding': 170,
         'Scan Port OS': 170,
         'Mirai-Hostbruteforceg': 170,
         'Mirai-UDP Flooding': 170,
         'Mirai-HTTP Flooding': 170,
         'Scan Hostport': 170,
         'MITM ARP Spoofing': 170,
         'Normal_I': 170})

In [19]:
from sklearn import preprocessing 
from sklearn.preprocessing import LabelEncoder
le = preprocessing.LabelEncoder()

dataIFeatures=dataIFeatures.apply(LabelEncoder().fit_transform)

In [20]:
dataFFeatures=dataFFeatures.apply(LabelEncoder().fit_transform)

In [21]:
len(dataIFeatures)

1530

In [22]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import PCA



X =dataIFeatures
X = X.astype(int)
y =  DataITarget["Sub_Cat"]

  
# Two features with highest chi-squared statistics are selected
chi2_features = SelectKBest(chi2, k = 30)
GF = chi2_features.fit_transform(X, y)

In [23]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.decomposition import PCA



X =dataFFeatures
X = X.astype(int)
y =  DataFTarget["Sub_Cat"]

  
# Two features with highest chi-squared statistics are selected
chi2_features = SelectKBest(chi2, k = 30)
GF1 = chi2_features.fit_transform(X, y)

In [24]:
dI=pd.DataFrame(GF)
dF=pd.DataFrame(GF1)

In [25]:
ComFeat=dF.append(dI,ignore_index = True) 
ComT=DataFTarget["Sub_Cat"].append(DataITarget["Sub_Cat"],ignore_index = True)

In [26]:
Counter(ComT)

Counter({'Normal': 170,
         'Backdoor': 170,
         'Analysis': 170,
         'Fuzzers': 170,
         'Shellcode': 170,
         'Reconnaissance': 170,
         'Exploits': 170,
         'DoS': 170,
         'Worms': 170,
         'Generic': 170,
         'Mirai-Ackflooding': 170,
         'DoS-Synflooding': 170,
         'Scan Port OS': 170,
         'Mirai-Hostbruteforceg': 170,
         'Mirai-UDP Flooding': 170,
         'Mirai-HTTP Flooding': 170,
         'Scan Hostport': 170,
         'MITM ARP Spoofing': 170,
         'Normal_I': 170})

In [27]:
feat=np.array(ComFeat)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(feat, ComT, test_size=0.15, random_state=1, stratify=ComT, shuffle=True)

In [30]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
a =  RandomForestClassifier(n_estimators=200, random_state=2,max_depth=150) 
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))


0.7896907216494845
                       precision    recall  f1-score   support

             Analysis       0.65      0.50      0.57        26
             Backdoor       0.74      0.77      0.75        26
                  DoS       0.75      0.84      0.79        25
      DoS-Synflooding       1.00      1.00      1.00        25
             Exploits       0.88      0.84      0.86        25
              Fuzzers       1.00      0.96      0.98        26
              Generic       1.00      1.00      1.00        25
    MITM ARP Spoofing       0.89      0.96      0.92        25
    Mirai-Ackflooding       0.25      0.23      0.24        26
  Mirai-HTTP Flooding       0.33      0.42      0.37        26
Mirai-Hostbruteforceg       0.96      0.88      0.92        25
   Mirai-UDP Flooding       0.90      0.69      0.78        26
               Normal       0.96      1.00      0.98        26
             Normal_I       0.87      1.00      0.93        26
       Reconnaissance       0.88   

In [31]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
a =  ExtraTreesClassifier(n_estimators=200, random_state=2,max_depth=150) 
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

0.7752577319587629
                       precision    recall  f1-score   support

             Analysis       0.57      0.46      0.51        26
             Backdoor       0.70      0.73      0.72        26
                  DoS       0.74      0.80      0.77        25
      DoS-Synflooding       1.00      1.00      1.00        25
             Exploits       0.81      0.84      0.82        25
              Fuzzers       0.96      0.96      0.96        26
              Generic       1.00      1.00      1.00        25
    MITM ARP Spoofing       0.85      0.92      0.88        25
    Mirai-Ackflooding       0.38      0.35      0.36        26
  Mirai-HTTP Flooding       0.39      0.50      0.44        26
Mirai-Hostbruteforceg       0.82      0.72      0.77        25
   Mirai-UDP Flooding       0.95      0.69      0.80        26
               Normal       0.96      1.00      0.98        26
             Normal_I       0.83      0.96      0.89        26
       Reconnaissance       0.88   

In [32]:
from sklearn.linear_model import LogisticRegression
a=LogisticRegression(random_state=1000, solver='sag',multi_class='multinomial',C=2.0)
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

0.6474226804123712
                       precision    recall  f1-score   support

             Analysis       0.43      0.38      0.41        26
             Backdoor       0.77      0.92      0.84        26
                  DoS       0.52      0.92      0.67        25
      DoS-Synflooding       1.00      0.96      0.98        25
             Exploits       0.73      0.44      0.55        25
              Fuzzers       0.70      0.73      0.72        26
              Generic       0.83      1.00      0.91        25
    MITM ARP Spoofing       0.56      0.60      0.58        25
    Mirai-Ackflooding       0.30      0.31      0.30        26
  Mirai-HTTP Flooding       0.32      0.35      0.33        26
Mirai-Hostbruteforceg       0.42      0.56      0.48        25
   Mirai-UDP Flooding       0.86      0.73      0.79        26
               Normal       1.00      0.96      0.98        26
             Normal_I       0.91      0.77      0.83        26
       Reconnaissance       0.81   

C:\Users\Sohail_Comp\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [33]:
from sklearn.neighbors import KNeighborsClassifier
a = KNeighborsClassifier(n_neighbors=5)
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

0.7587628865979381
                       precision    recall  f1-score   support

             Analysis       0.52      0.42      0.47        26
             Backdoor       0.68      0.73      0.70        26
                  DoS       0.64      0.84      0.72        25
      DoS-Synflooding       1.00      0.96      0.98        25
             Exploits       0.74      0.68      0.71        25
              Fuzzers       0.89      0.92      0.91        26
              Generic       0.96      1.00      0.98        25
    MITM ARP Spoofing       0.89      0.96      0.92        25
    Mirai-Ackflooding       0.39      0.42      0.41        26
  Mirai-HTTP Flooding       0.43      0.46      0.44        26
Mirai-Hostbruteforceg       0.83      0.80      0.82        25
   Mirai-UDP Flooding       0.82      0.69      0.75        26
               Normal       0.96      0.92      0.94        26
             Normal_I       0.92      0.92      0.92        26
       Reconnaissance       0.76   

In [34]:
from sklearn.svm import SVC
a=SVC(kernel='linear', C=2.0)
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

0.7443298969072165
                       precision    recall  f1-score   support

             Analysis       0.52      0.46      0.49        26
             Backdoor       0.74      0.96      0.83        26
                  DoS       0.79      0.88      0.83        25
      DoS-Synflooding       1.00      0.96      0.98        25
             Exploits       0.85      0.68      0.76        25
              Fuzzers       1.00      0.85      0.92        26
              Generic       0.89      1.00      0.94        25
    MITM ARP Spoofing       0.74      0.80      0.77        25
    Mirai-Ackflooding       0.47      0.77      0.58        26
  Mirai-HTTP Flooding       0.45      0.35      0.39        26
Mirai-Hostbruteforceg       0.47      0.60      0.53        25
   Mirai-UDP Flooding       1.00      0.65      0.79        26
               Normal       1.00      1.00      1.00        26
             Normal_I       1.00      0.92      0.96        26
       Reconnaissance       0.92   

In [35]:
X_train, X_test, y_train, y_test = train_test_split(GF, DataITarget["Sub_Cat"], test_size=0.15, random_state=1, stratify=DataITarget["Sub_Cat"], shuffle=True)

In [36]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
a =  RandomForestClassifier(n_estimators=200, random_state=2,max_depth=150) 
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))


0.7608695652173914
                       precision    recall  f1-score   support

      DoS-Synflooding       1.00      1.00      1.00        26
    MITM ARP Spoofing       0.93      0.96      0.94        26
    Mirai-Ackflooding       0.31      0.36      0.33        25
  Mirai-HTTP Flooding       0.37      0.38      0.38        26
Mirai-Hostbruteforceg       0.96      0.88      0.92        26
   Mirai-UDP Flooding       0.89      0.65      0.76        26
             Normal_I       0.92      0.96      0.94        25
        Scan Hostport       0.81      0.84      0.82        25
         Scan Port OS       0.77      0.80      0.78        25

             accuracy                           0.76       230
            macro avg       0.77      0.76      0.76       230
         weighted avg       0.77      0.76      0.77       230

[[26  0  0  0  0  0  0  0  0]
 [ 0 25  0  0  0  0  1  0  0]
 [ 0  1  9 14  0  1  0  0  0]
 [ 0  1 14 10  0  1  0  0  0]
 [ 0  0  0  0 23  0  1  0  2]
 [ 0  0  

In [37]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
a =  ExtraTreesClassifier(n_estimators=200, random_state=2,max_depth=150) 
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

0.7347826086956522
                       precision    recall  f1-score   support

      DoS-Synflooding       1.00      1.00      1.00        26
    MITM ARP Spoofing       0.90      1.00      0.95        26
    Mirai-Ackflooding       0.37      0.44      0.40        25
  Mirai-HTTP Flooding       0.42      0.38      0.40        26
Mirai-Hostbruteforceg       0.92      0.92      0.92        26
   Mirai-UDP Flooding       0.89      0.65      0.76        26
             Normal_I       0.96      0.96      0.96        25
        Scan Hostport       0.57      0.80      0.67        25
         Scan Port OS       0.69      0.44      0.54        25

             accuracy                           0.73       230
            macro avg       0.75      0.73      0.73       230
         weighted avg       0.75      0.73      0.73       230

[[26  0  0  0  0  0  0  0  0]
 [ 0 26  0  0  0  0  0  0  0]
 [ 0  2 11 10  1  1  0  0  0]
 [ 0  1 14 10  0  1  0  0  0]
 [ 0  0  0  0 24  0  1  1  0]
 [ 0  0  

In [38]:
from sklearn.linear_model import LogisticRegression
a=LogisticRegression(random_state=1000, solver='sag',multi_class='multinomial',C=2.0)
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

0.5608695652173913
                       precision    recall  f1-score   support

      DoS-Synflooding       1.00      1.00      1.00        26
    MITM ARP Spoofing       0.44      0.58      0.50        26
    Mirai-Ackflooding       0.33      0.24      0.28        25
  Mirai-HTTP Flooding       0.40      0.46      0.43        26
Mirai-Hostbruteforceg       0.45      0.50      0.47        26
   Mirai-UDP Flooding       0.86      0.69      0.77        26
             Normal_I       0.95      0.80      0.87        25
        Scan Hostport       0.31      0.20      0.24        25
         Scan Port OS       0.40      0.56      0.47        25

             accuracy                           0.56       230
            macro avg       0.57      0.56      0.56       230
         weighted avg       0.57      0.56      0.56       230

[[26  0  0  0  0  0  0  0  0]
 [ 0 15  1  0  6  0  0  2  2]
 [ 0  5  6 11  0  2  1  0  0]
 [ 0  2 10 12  1  1  0  0  0]
 [ 0  4  0  1 13  0  0  3  5]
 [ 0  1  

C:\Users\Sohail_Comp\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [39]:
from sklearn.neighbors import KNeighborsClassifier
a = KNeighborsClassifier(n_neighbors=5)
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

0.7086956521739131
                       precision    recall  f1-score   support

      DoS-Synflooding       1.00      1.00      1.00        26
    MITM ARP Spoofing       0.89      0.96      0.93        26
    Mirai-Ackflooding       0.38      0.44      0.41        25
  Mirai-HTTP Flooding       0.35      0.35      0.35        26
Mirai-Hostbruteforceg       0.90      0.73      0.81        26
   Mirai-UDP Flooding       0.85      0.65      0.74        26
             Normal_I       0.88      0.92      0.90        25
        Scan Hostport       0.59      0.76      0.67        25
         Scan Port OS       0.64      0.56      0.60        25

             accuracy                           0.71       230
            macro avg       0.72      0.71      0.71       230
         weighted avg       0.72      0.71      0.71       230

[[26  0  0  0  0  0  0  0  0]
 [ 0 25  0  0  1  0  0  0  0]
 [ 0  1 11 11  0  2  0  0  0]
 [ 0  1 14  9  0  1  1  0  0]
 [ 0  1  0  1 19  0  2  2  1]
 [ 0  0  

In [40]:
from sklearn.svm import SVC
a=SVC(kernel='linear', C=2.0)
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

0.6130434782608696
                       precision    recall  f1-score   support

      DoS-Synflooding       1.00      1.00      1.00        26
    MITM ARP Spoofing       0.62      0.69      0.65        26
    Mirai-Ackflooding       0.35      0.36      0.35        25
  Mirai-HTTP Flooding       0.38      0.50      0.43        26
Mirai-Hostbruteforceg       0.52      0.65      0.58        26
   Mirai-UDP Flooding       1.00      0.65      0.79        26
             Normal_I       0.96      0.96      0.96        25
        Scan Hostport       0.38      0.32      0.35        25
         Scan Port OS       0.47      0.36      0.41        25

             accuracy                           0.61       230
            macro avg       0.63      0.61      0.61       230
         weighted avg       0.63      0.61      0.62       230

[[26  0  0  0  0  0  0  0  0]
 [ 0 18  0  2  5  0  0  1  0]
 [ 0  4  9 12  0  0  0  0  0]
 [ 0  0 13 13  0  0  0  0  0]
 [ 0  4  1  2 17  0  1  1  0]
 [ 0  1  

In [28]:
X_train, X_test, y_train, y_test = train_test_split(GF1, DataFTarget["Sub_Cat"], test_size=0.15, random_state=1, stratify=DataFTarget["Sub_Cat"], shuffle=True)

In [42]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
a =  RandomForestClassifier(n_estimators=200, random_state=2,max_depth=150) 
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))


0.807843137254902
                precision    recall  f1-score   support

      Analysis       0.50      0.48      0.49        25
      Backdoor       0.67      0.69      0.68        26
           DoS       0.68      0.84      0.75        25
      Exploits       0.90      0.73      0.81        26
       Fuzzers       1.00      0.92      0.96        25
       Generic       1.00      1.00      1.00        26
        Normal       0.96      1.00      0.98        25
Reconnaissance       0.81      0.65      0.72        26
     Shellcode       0.80      0.77      0.78        26
         Worms       0.81      1.00      0.89        25

      accuracy                           0.81       255
     macro avg       0.81      0.81      0.81       255
  weighted avg       0.81      0.81      0.81       255

[[12  9  3  0  0  0  0  1  0  0]
 [ 7 18  0  0  0  0  1  0  0  0]
 [ 2  0 21  0  0  0  0  2  0  0]
 [ 1  0  6 19  0  0  0  0  0  0]
 [ 1  0  0  0 23  0  0  0  0  1]
 [ 0  0  0  0  0 26  0  0  0  

In [43]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
a =  ExtraTreesClassifier(n_estimators=200, random_state=2,max_depth=150) 
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

0.796078431372549
                precision    recall  f1-score   support

      Analysis       0.43      0.36      0.39        25
      Backdoor       0.60      0.69      0.64        26
           DoS       0.70      0.84      0.76        25
      Exploits       0.90      0.73      0.81        26
       Fuzzers       0.96      0.88      0.92        25
       Generic       1.00      1.00      1.00        26
        Normal       0.96      1.00      0.98        25
Reconnaissance       0.81      0.65      0.72        26
     Shellcode       0.78      0.81      0.79        26
         Worms       0.83      1.00      0.91        25

      accuracy                           0.80       255
     macro avg       0.80      0.80      0.79       255
  weighted avg       0.80      0.80      0.79       255

[[ 9 12  3  0  0  0  0  1  0  0]
 [ 7 18  0  0  0  0  1  0  0  0]
 [ 2  0 21  0  0  0  0  2  0  0]
 [ 1  0  5 19  1  0  0  0  0  0]
 [ 1  0  0  1 22  0  0  0  0  1]
 [ 0  0  0  0  0 26  0  0  0  

In [29]:
from sklearn.linear_model import LogisticRegression
a=LogisticRegression(random_state=1000, solver='sag',multi_class='multinomial',C=2.0)
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

0.6784313725490196
                precision    recall  f1-score   support

      Analysis       0.41      0.36      0.38        25
      Backdoor       0.72      0.88      0.79        26
           DoS       0.56      0.92      0.70        25
      Exploits       0.50      0.31      0.38        26
       Fuzzers       0.67      0.56      0.61        25
       Generic       0.96      1.00      0.98        26
        Normal       0.96      0.96      0.96        25
Reconnaissance       0.75      0.46      0.57        26
     Shellcode       0.62      0.62      0.62        26
         Worms       0.62      0.72      0.67        25

      accuracy                           0.68       255
     macro avg       0.68      0.68      0.67       255
  weighted avg       0.68      0.68      0.67       255

[[ 9  8  4  3  0  0  0  0  0  1]
 [ 3 23  0  0  0  0  0  0  0  0]
 [ 1  0 23  0  0  0  0  0  1  0]
 [ 5  0  9  8  1  1  0  0  1  1]
 [ 3  1  0  2 14  0  0  2  0  3]
 [ 0  0  0  0  0 26  0  0  0 

C:\Users\Sohail_Comp\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [44]:
from sklearn.neighbors import KNeighborsClassifier
a = KNeighborsClassifier(n_neighbors=5)
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

0.7607843137254902
                precision    recall  f1-score   support

      Analysis       0.59      0.52      0.55        25
      Backdoor       0.71      0.77      0.74        26
           DoS       0.58      0.84      0.69        25
      Exploits       0.68      0.65      0.67        26
       Fuzzers       0.88      0.88      0.88        25
       Generic       1.00      1.00      1.00        26
        Normal       0.96      0.96      0.96        25
Reconnaissance       0.79      0.58      0.67        26
     Shellcode       0.71      0.65      0.68        26
         Worms       0.76      0.76      0.76        25

      accuracy                           0.76       255
     macro avg       0.77      0.76      0.76       255
  weighted avg       0.77      0.76      0.76       255

[[13  8  2  1  0  0  0  1  0  0]
 [ 5 20  0  1  0  0  0  0  0  0]
 [ 1  0 21  1  0  0  0  2  0  0]
 [ 0  0  9 17  0  0  0  0  0  0]
 [ 1  0  0  2 22  0  0  0  0  0]
 [ 0  0  0  0  0 26  0  0  0 

In [45]:
from sklearn.svm import SVC
a=SVC(kernel='linear', C=2.0)
a_pred = a.fit(X_train, y_train)
pred=a_pred.predict(X_test)
print(accuracy_score(y_test,pred))
print(classification_report(y_test,pred))
print(confusion_matrix(y_test,pred))

0.7490196078431373
                precision    recall  f1-score   support

      Analysis       0.48      0.48      0.48        25
      Backdoor       0.74      0.88      0.81        26
           DoS       0.59      0.88      0.71        25
      Exploits       0.65      0.42      0.51        26
       Fuzzers       0.95      0.84      0.89        25
       Generic       1.00      1.00      1.00        26
        Normal       0.96      1.00      0.98        25
Reconnaissance       0.80      0.62      0.70        26
     Shellcode       0.67      0.54      0.60        26
         Worms       0.70      0.84      0.76        25

      accuracy                           0.75       255
     macro avg       0.75      0.75      0.74       255
  weighted avg       0.75      0.75      0.74       255

[[12  8  3  1  0  0  0  1  0  0]
 [ 3 23  0  0  0  0  0  0  0  0]
 [ 2  0 22  0  0  0  0  0  1  0]
 [ 7  0  7 11  0  0  0  0  0  1]
 [ 0  0  0  1 21  0  0  0  0  3]
 [ 0  0  0  0  0 26  0  0  0 